In [ ]:
!pip install tensorflow numpy pandas yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Download historical stock data
ticker = 'ADANIENSOL.NS'
data = yf.download(ticker)
prices = data['Close'].values

# Normalize the data
scaler = MinMaxScaler()
prices = scaler.fit_transform(prices.reshape(-1, 1))

# Prepare sequences for the Transformer model
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 10  
X, y = create_sequences(prices, seq_length)

# Split the data into training and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape X for the Transformer
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

[*********************100%%**********************]  1 of 1 completed


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention
from tensorflow.keras.optimizers import Adam

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.time_emb = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(embed_dim))

    def call(self, x):
        maxlen = tf.shape(x)[-2]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.time_emb(x)
        return x + positions

def create_transformer_model(seq_length, feature_dim, embed_dim, num_heads, ff_dim):
    inputs = Input(shape=(seq_length, feature_dim))
    embedding_layer = TokenAndPositionEmbedding(seq_length, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = transformer_block(x)  # Stack multiple Transformer blocks
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(20, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(1)(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model hyperparameters
embed_dim = 64  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer

model = create_transformer_model(seq_length, 1, embed_dim, num_heads, ff_dim)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 10, 1)]              0         []                            
                                                                                                  
 token_and_position_embeddi  (None, 10, 64)               768       ['input_3[0][0]']             
 ng_2 (TokenAndPositionEmbe                                                                       
 dding)                                                                                           
                                                                                                  
 transformer_block_2 (Trans  (None, 10, 64)               74944     ['token_and_position_embedding
 formerBlock)                                                       _2[0][0]',              

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
6/6 [==============================] - 9s 310ms/step - loss: 0.3767 - val_loss: 0.0131
Epoch 2/50
6/6 [==============================] - 0s 58ms/step - loss: 0.0999 - val_loss: 0.0805
Epoch 3/50
6/6 [==============================] - 0s 76ms/step - loss: 0.0993 - val_loss: 0.0175
Epoch 4/50
6/6 [==============================] - 0s 67ms/step - loss: 0.0798 - val_loss: 0.0212
Epoch 5/50
6/6 [==============================] - 0s 58ms/step - loss: 0.0651 - val_loss: 0.0246
Epoch 6/50
6/6 [==============================] - 0s 55ms/step - loss: 0.0660 - val_loss: 0.0087
Epoch 7/50
6/6 [==============================] - 0s 62ms/step - loss: 0.0555 - val_loss: 0.0102
Epoch 8/50
6/6 [==============================] - 0s 68ms/step - loss: 0.0571 - val_loss: 0.0217
Epoch 9/50
6/6 [==============================] - 0s 67ms/step - loss: 0.0408 - val_loss: 0.0116
Epoch 10/50
6/6 [==============================] - 0s 56ms/step - loss: 0.0430 - val_loss: 0.0091
Epoch 11/50
6/6 [===========

In [ ]:
# Function to predict future steps
def predict_future_steps(model, data, steps):
    predictions = []
    input_seq = data[-seq_length:].reshape((1, seq_length, 1))
    for _ in range(steps):
        pred = model.predict(input_seq)
        predictions.append(pred[0, 0])
        input_seq = np.append(input_seq[:, 1:, :], pred.reshape(1, 1, 1), axis=1)
    return np.array(predictions)

# Predict the next day and week prices
next_1d_pred = predict_future_steps(model, prices, 1)
next_1w_pred = predict_future_steps(model, prices, 7)

# Transform the predictions back to the original scale
next_1d_pred = scaler.inverse_transform(next_1d_pred.reshape(-1, 1))
next_1w_pred = scaler.inverse_transform(next_1w_pred.reshape(-1, 1))

1/1 [==============================] - 0s 55ms/step


In [ ]:
print(f'1-Day Forecast: {next_1d_pred.flatten()}')
print(f'1-Week Forecast: {next_1w_pred.flatten()}')

1-Day Forecast: [1039.5013]
1-Week Forecast: [1039.5013  1031.773   1025.6685  1020.6773  1017.29626 1014.89636
 1013.10284]
